In [196]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
import os

In [ ]:
#La siguiente funcion lee las 3 carpetas de informacion y crea 3 dataframes, luego a cada uno les realiza la ventana movil
#para asi fusionarlos en un solo dataframe grande. Despues se escala una base solo con las variables numericas para finalmente 
#realizar los primeros modelos (regresion lineal y logistica).

In [276]:
def probando(file1,file2,file3,delta_movil):
    
    ######Cargar bases NORMAL, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_normal_files=os.listdir(file1)
    #Consolidando todas las bases con comportamiento normal en un DF
    consolidated_normal_df = pd.DataFrame()
    for name in names_normal_files:
        filename= f'df_{name[:-4]}'
        path = F'{file1}/{name}'
        filename = pd.read_csv(path)
        consolidated_normal_df = pd.concat([consolidated_normal_df,filename])
    
    #Asignando etiquetas
    consolidated_normal_df['Label']=0
    consolidated_normal_df['dataset']="Normal"
    
    #Ventana movil
    data_auxN=pd.DataFrame.copy(consolidated_normal_df)
    data_auxN['Time'] = data_auxN.index if data_auxN.index.is_monotonic_increasing else range(len(data_auxN))
    data_auxN.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxN2 = pd.DataFrame.copy(consolidated_normal_df)
        data_auxN2['Time']= (data_auxN2.index if data_auxN2.index.is_monotonic_increasing else range(len(data_auxN2)))
        data_auxN2['Time']=data_auxN2['Time']+j
        data_auxN2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j), 'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxN3 = data_auxN.merge(data_auxN2, on='Time', how='inner')
        data_auxN = pd.DataFrame.copy(data_auxN3)
    #return data_auxN

    
    ######Cargar bases ATAQUE, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_attacked_files=os.listdir(file2) 
    #Consolidando todas las bases con comportamiento atacado en un DF
    consolidated_attacked_df = pd.DataFrame()
    for name in names_attacked_files:
        filename= f'df_{name[:-4]}'
        path = f'Attacked_Operation/{name}'
        filename = pd.read_csv(path)
        consolidated_attacked_df = pd.concat([consolidated_attacked_df,filename])
    
    #Funcion para etiquetar si la medición esta bajo ataque o no (sirve para este item como para las bases desbalanceadas)
    def label_data(time):
        if time <= 0.3:
            return 0
        else:
            return 1
    consolidated_attacked_df['Label']=consolidated_attacked_df['Time'].apply(label_data)

    #Por temas de testeo, no necesitamos datos despues de los 0,5 segundos por lo que deben ser removidos.
    consolidated_attacked_df.reset_index(drop=True,inplace=True)
    boolean_mask = consolidated_attacked_df['Time']>0.5
    attacked_index = consolidated_attacked_df[boolean_mask].index
    consolidated_attacked_df.drop(attacked_index, axis=0,inplace=True)
    
    #Ahora etiqueto que es una base con ataque
    consolidated_attacked_df['dataset'] = "Attacked"

    #Ventana móvil
    data_auxA=pd.DataFrame.copy(consolidated_attacked_df)
    data_auxA['Time'] = data_auxA.index if data_auxA.index.is_monotonic_increasing else range(len(data_auxA))
    data_auxA.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxA2 = pd.DataFrame.copy(consolidated_attacked_df)
        data_auxA2['Time']= (data_auxA2.index if data_auxA2.index.is_monotonic_increasing else range(len(data_auxA2)))
        data_auxA2['Time']=data_auxA2['Time']+j
        data_auxA2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j),'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxA3 = data_auxA.merge(data_auxA2, on='Time', how='inner')
        data_auxA = pd.DataFrame.copy(data_auxA3)
    #return data_auxA
    
    ######Cargar bases DESBALANCEADAS ATACADAS, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_imbalanced_files=os.listdir(file3)
    #Consolidando todas las bases desbalanceadas con comportamiento atacado en un DF
    consolidated_imbalanced_df = pd.DataFrame()
    for name in names_imbalanced_files:
        filename= f'df_{name[:-4]}'
        path = f'Imbalanced_Attacked/{name}'
        filename = pd.read_csv(path)
        consolidated_imbalanced_df = pd.concat([consolidated_imbalanced_df,filename])
    
    #Aplico etiqueta si esta bajo ataque o no
    consolidated_imbalanced_df['Label']=consolidated_imbalanced_df['Time'].apply(label_data)
    
    #Por temas de testeo, no necesitamos datos despues de los 0,5 segundos por lo que deben ser removidos.
    consolidated_imbalanced_df.reset_index(drop=True,inplace=True)
    boolean_im_mask = consolidated_imbalanced_df['Time']>0.5
    imbalanced_index = consolidated_imbalanced_df[boolean_im_mask].index
    consolidated_imbalanced_df.drop(imbalanced_index,axis=0,inplace=True)
    
    #Ahora etiqueto que es una base desbalanceada con ataque
    consolidated_imbalanced_df['dataset'] = "Imbalanced"
    
    #Ventana móvil
    data_auxIA=pd.DataFrame.copy(consolidated_imbalanced_df)
    data_auxIA['Time'] = data_auxIA.index if data_auxIA.index.is_monotonic_increasing else range(len(data_auxIA))
    data_auxIA.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxIA2 = pd.DataFrame.copy(consolidated_imbalanced_df)
        data_auxIA2['Time']= (data_auxIA2.index if data_auxIA2.index.is_monotonic_increasing else range(len(data_auxIA2)))
        data_auxIA2['Time']=data_auxIA2['Time']+j
        data_auxIA2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j),'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxIA3 = data_auxIA.merge(data_auxIA2, on='Time', how='inner')
        data_auxIA = pd.DataFrame.copy(data_auxIA3)
    #return data_auxIA    

    ###### Consolidando todos los dataframes, con ventana de tiempo, en uno--------------------------------------------------------
    consolidated_df = pd.concat([data_auxN, data_auxA]) #bases normales con atacadas
    consolidated_df = pd.concat([consolidated_df, data_auxIA]) #lo anterior + desbalanceado
    #print(consolidated_df.Label.value_counts())
    
    #####Reseteando los indices
    consolidated_df.reset_index(drop=True,inplace=True)
    #print(consolidated_df)
    
    
    #ESTO ES LA PRUEBA TRATANDO DE ESCALAR, EN LA SIGUIENTE LINEA DE CODIGO VEREMOS QUE PASA CON EL MODELO SIN ESCALAR.
    
    
    ##### Preporcesamiento de la data
    #Importando para escalar
    from sklearn.preprocessing import StandardScaler
    scaler=StandardScaler()
    
    #Solo se necesita escalar las variables numericas, asi que se realiza eso
    numerical_consolidated_df=consolidated_df.drop(['Time','Label','dataset'],axis=1)
    
    numerical_consolidated_df2=pd.DataFrame.copy(numerical_consolidated_df)
    
    for j in range(1,delta_movil+1):
        numerical_consolidated_df2=numerical_consolidated_df2.drop(['dataset_'+str(j),'Label_'+str(j)],axis=1)
    #return numerical_consolidated_df2
    
    #Escalar solo los valores numericos y colocar de vuelta a un df lo que era string
    scaled_consolidated_df=scaler.fit_transform(numerical_consolidated_df2)
    scaled_consolidated_df = pd.DataFrame(scaled_consolidated_df,columns=numerical_consolidated_df2.columns)
    #print(scaled_consolidated_df)
    
    #Volver a concatenar las columnas de string
    consolidated_df2=pd.concat([scaled_consolidated_df,consolidated_df[['Time','Label','dataset']]],axis=1)
    #print(consolidated_df2)
    for j in range(1,delta_movil+1):
        consolidated_df2=pd.concat([consolidated_df2,consolidated_df[['dataset_'+str(j),'Label_'+str(j)]]],axis=1)
    #return consolidated_df2
    
    
    ######### Testing First models -------------------------------------------------------------------------------------------
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import cross_val_score
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.model_selection import train_test_split
    
    #Analizando un poco de forma grafica:
    #fig, ax = plt.subplots(figsize=(6, 3.84))
    #sns.violinplot(
    #    x     = 'Label',
     #   y     = 'V3',
      #  data  = consolidated_df2,
        #color = "white",
       # ax = ax
    #)

    #ax.set_title('probando');
    from statsmodels.stats.weightstats import ttest_ind
    res_ttest = ttest_ind(
                    x1 = consolidated_df2.V3[consolidated_df2.Label == 0],
                    x2 = consolidated_df2.V3[consolidated_df2.Label == 1],
                    alternative='two-sided'
                )
    print("Para entrenamiento de los datos con regresión logística se tienen los siguientes resultados:")
    print(f"t={res_ttest[0]}, p-value={res_ttest[1]}") #viendo el valor t y p
    
    ##### Train, test, split the data -----------------------------------------------------------------------------------------
    X=scaled_consolidated_df 
    y=consolidated_df2['Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)
    
    ##Regresion LOGISTICA:
    
    #Creacion y entrenamiento del modelo logístico
    reg_log = LogisticRegression() #cargo el modelo de reg log
    reg_log.fit(X_train, y_train) #Entreno el clasificador
    
    print("Valor del intercepto:", reg_log.intercept_)
    #print("Valores de los Coeficientes:", list(zip(X.columns, reg_log.coef_.flatten(), )))
    print(f"Accuracy de entrenamiento para Reg Log: {100*(reg_log.score(X, y))}%")
    
    #Prediccion
    y_pred = reg_log.predict(X_train) #guardo las predicciones con el modelo entrenado
    
    print("")
    print("Para testeo de los datos con regresión logística se tienen los siguientes resultados:")
    #Accuracy de testeo
    #X_test = sm.add_constant(X_test, prepend=True) #no se en realidad si esto se debe agregar o no
    predicciones = reg_log.predict(X_test)
    clasificacion = np.where(predicciones<0.5, 0, 1)
    accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    = clasificacion,
            normalize = True
           )
    print(f"El accuracy de testeo en Reg Log es: {100*accuracy}%")
    print("")
    
    
    ##RANDOM FOREST:
    
    #Creacion y entrenamiento del modelo Random Forest
    random_forest = RandomForestRegressor(
            n_estimators = 100,
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,
            random_state = 100
         )

    random_forest.fit(X_train, y_train)
    #Error cuadratico medio - entrenamiento
    pred_entr=random_forest.predict(X=X_train)
    rmse_entr = mean_squared_error(
        y_true  = y_train,
        y_pred  = pred_entr,
        squared = False
       )
    print("Para entrenamiento de los datos con random forest:")
    print(f"El error (rmse) de entrenamiento es: {rmse_entr}")
    #print(f"Accuracy de entrenamiento para Random Forest: {100*(random_forest.score(X, y))}%")
    
    #Error cuadratico medio - testeo
    pred_test = random_forest.predict(X = X_test)

    rmse_test = mean_squared_error(
        y_true  = y_test,
        y_pred  = pred_test,
        squared = False
       )
    print("Para testeo de los datos con random forest:")
    print(f"El error (rmse) de test es: {rmse_test}")
    
    
    ## SUPPORT VECTOR MACHINE (SVM)
    
    
    
    #AAAAAAAAAAAAAAAAAAAAAAAA
    
    
    ## K MEANS:
    #KNN_model = KNeighborsClassifier(n_neighbors=5)
    #KNN_model.fit(X_train, y_train)
    #KNN_prediction = KNN_model.predict(X_test)
    
    #Accuracy score is the simplest way to evaluate
    #print(accuracy_score(y_pred, y_test))
    #Confusion Matrix and Classification Report give more details about performance
    #print(classification_report(y_pred, y_test))
    
    #matrix=confusion_matrix(y_test,KNN_prediction)
    #ax= plt.subplot()
    #sns.heatmap(matrix, annot=True, fmt='g', ax=ax);
    # labels, title and ticks
    #ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    #ax.set_title('Confusion Matrix'); 
    #ax.xaxis.set_ticklabels(['attack', 'normal']); ax.yaxis.set_ticklabels(['normal', 'attack']);
    
    
    
    
probando("Normal_Operation","Attacked_Operation","Imbalanced_Attacked",2)  

Para entrenamiento de los datos con regresión logística se tienen los siguientes resultados:
t=134.9964450850342, p-value=0.0
Valor del intercepto: [-1.53153775]
Accuracy de entrenamiento para Reg Log: 99.4718554551772%

Para testeo de los datos con regresión logística se tienen los siguientes resultados:
El accuracy de testeo en Reg Log es: 99.30507296733843%

Para entrenamiento de los datos con random forest:
El error (rmse) de entrenamiento es: 0.007308900994923842
Para testeo de los datos con random forest:
El error (rmse) de test es: 0.030557706945832052


In [ ]:
#El codigo a continuacion es la misma funcion de arriba pero sin escalar la base numerica

In [148]:
def probando(file1,file2,file3,delta_movil):
    
    ######Cargar bases NORMAL, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_normal_files=os.listdir(file1)
    #Consolidando todas las bases con comportamiento normal en un DF
    consolidated_normal_df = pd.DataFrame()
    for name in names_normal_files:
        filename= f'df_{name[:-4]}'
        path = F'{file1}/{name}'
        filename = pd.read_csv(path)
        consolidated_normal_df = pd.concat([consolidated_normal_df,filename])
    
    #Asignando etiquetas
    consolidated_normal_df['Label']=0
    consolidated_normal_df['dataset']="Normal"
    
    #Ventana movil
    data_auxN=pd.DataFrame.copy(consolidated_normal_df)
    data_auxN['Time'] = data_auxN.index if data_auxN.index.is_monotonic_increasing else range(len(data_auxN))
    data_auxN.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxN2 = pd.DataFrame.copy(consolidated_normal_df)
        data_auxN2['Time']= (data_auxN2.index if data_auxN2.index.is_monotonic_increasing else range(len(data_auxN2)))
        data_auxN2['Time']=data_auxN2['Time']+j
        data_auxN2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j), 'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxN3 = data_auxN.merge(data_auxN2, on='Time', how='inner')
        data_auxN = pd.DataFrame.copy(data_auxN3)
    #return data_auxN

    
    ######Cargar bases ATAQUE, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_attacked_files=os.listdir(file2) 
    #Consolidando todas las bases con comportamiento atacado en un DF
    consolidated_attacked_df = pd.DataFrame()
    for name in names_attacked_files:
        filename= f'df_{name[:-4]}'
        path = f'Attacked_Operation/{name}'
        filename = pd.read_csv(path)
        consolidated_attacked_df = pd.concat([consolidated_attacked_df,filename])
    
    #Funcion para etiquetar si la medición esta bajo ataque o no (sirve para este item como para las bases desbalanceadas)
    def label_data(time):
        if time <= 0.3:
            return 0
        else:
            return 1
    consolidated_attacked_df['Label']=consolidated_attacked_df['Time'].apply(label_data)

    #Por temas de testeo, no necesitamos datos despues de los 0,5 segundos por lo que deben ser removidos.
    consolidated_attacked_df.reset_index(drop=True,inplace=True)
    boolean_mask = consolidated_attacked_df['Time']>0.5
    attacked_index = consolidated_attacked_df[boolean_mask].index
    consolidated_attacked_df.drop(attacked_index, axis=0,inplace=True)
    
    #Ahora etiqueto que es una base con ataque
    consolidated_attacked_df['dataset'] = "Attacked"

    #Ventana móvil
    data_auxA=pd.DataFrame.copy(consolidated_attacked_df)
    data_auxA['Time'] = data_auxA.index if data_auxA.index.is_monotonic_increasing else range(len(data_auxA))
    data_auxA.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxA2 = pd.DataFrame.copy(consolidated_attacked_df)
        data_auxA2['Time']= (data_auxA2.index if data_auxA2.index.is_monotonic_increasing else range(len(data_auxA2)))
        data_auxA2['Time']=data_auxA2['Time']+j
        data_auxA2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j),'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxA3 = data_auxA.merge(data_auxA2, on='Time', how='inner')
        data_auxA = pd.DataFrame.copy(data_auxA3)
    #return data_auxA
    
    ######Cargar bases DESBALANCEADAS ATACADAS, asignar etiqueta y ventana de tiempo.---------------------------------------------------------------------
    names_imbalanced_files=os.listdir(file3)
    #Consolidando todas las bases desbalanceadas con comportamiento atacado en un DF
    consolidated_imbalanced_df = pd.DataFrame()
    for name in names_imbalanced_files:
        filename= f'df_{name[:-4]}'
        path = f'Imbalanced_Attacked/{name}'
        filename = pd.read_csv(path)
        consolidated_imbalanced_df = pd.concat([consolidated_imbalanced_df,filename])
    
    #Aplico etiqueta si esta bajo ataque o no
    consolidated_imbalanced_df['Label']=consolidated_imbalanced_df['Time'].apply(label_data)
    
    #Por temas de testeo, no necesitamos datos despues de los 0,5 segundos por lo que deben ser removidos.
    consolidated_imbalanced_df.reset_index(drop=True,inplace=True)
    boolean_im_mask = consolidated_imbalanced_df['Time']>0.5
    imbalanced_index = consolidated_imbalanced_df[boolean_im_mask].index
    consolidated_imbalanced_df.drop(imbalanced_index,axis=0,inplace=True)
    
    #Ahora etiqueto que es una base desbalanceada con ataque
    consolidated_imbalanced_df['dataset'] = "Imbalanced"
    
    #Ventana móvil
    data_auxIA=pd.DataFrame.copy(consolidated_imbalanced_df)
    data_auxIA['Time'] = data_auxIA.index if data_auxIA.index.is_monotonic_increasing else range(len(data_auxIA))
    data_auxIA.rename(columns={'Vm1:Measured voltage:1': 'V1', 'Vm1:Measured voltage:2':'V2', 
                                          'Vm1:Measured voltage:3':'V3','Divide18:1': 'IM1','Divide18:2':'IM2','Divide18:3': 'IM3',
                                          'Am1:Measured current': 'Corriente'}, inplace=True)
    for j in range(1,delta_movil+1):
        data_auxIA2 = pd.DataFrame.copy(consolidated_imbalanced_df)
        data_auxIA2['Time']= (data_auxIA2.index if data_auxIA2.index.is_monotonic_increasing else range(len(data_auxIA2)))
        data_auxIA2['Time']=data_auxIA2['Time']+j
        data_auxIA2.rename(columns={'Vm1:Measured voltage:1': 'V1_'+str(j), 'Vm1:Measured voltage:2':'V2_'+str(j), 
                                            'Vm1:Measured voltage:3':'V3_'+str(j),'Divide18:1': 'IM1_'+str(j),'Divide18:2':'IM2_'+str(j),
                                            'Divide18:3': 'IM3_'+str(j),'Am1:Measured current': 'Corriente_'+str(j),'Label': 'Label_'+str(j),
                                            'dataset': 'dataset_'+str(j)}, inplace=True)
        data_auxIA3 = data_auxIA.merge(data_auxIA2, on='Time', how='inner')
        data_auxIA = pd.DataFrame.copy(data_auxIA3)
    #return data_auxIA    

    ###### Consolidando todos los dataframes, con ventana de tiempo, en uno
    consolidated_df = pd.concat([data_auxN, data_auxA]) #bases normales con atacadas
    consolidated_df = pd.concat([consolidated_df, data_auxIA]) #lo anterior + desbalanceado
    print(consolidated_df.Label.value_counts())
    
    #####Reseteando los indices
    consolidated_df.reset_index(drop=True,inplace=True)
    #print(consolidated_df)
    
    
    #ESTO ES LA PRUEBA SIN ESCALAR
    
    numerical_consolidated_df=consolidated_df.drop(['Time','Label','dataset'],axis=1)
    base_aux=pd.DataFrame.copy(numerical_consolidated_df)
    
    for j in range(1,delta_movil+1):
        base_aux=base_aux.drop(['dataset_'+str(j),'Label_'+str(j)],axis=1)
    #return base_aux
    
    
    ##### Train, test, split the data
    X=base_aux
    y=consolidated_df['Label']
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Testing First models
    ## LINEAR MODELS
    from sklearn.linear_model import LinearRegression, LogisticRegression
    regr = LinearRegression()
    regr2=LogisticRegression()
    
    regr.fit(X_train, y_train)
    regr2.fit(X_train, y_train)
    
    y_pred = regr.predict(X_train)
    print('Coefficients: \n', regr.coef_)
    from sklearn.metrics import mean_squared_error, r2_score
    print("Mean squared error: %.2f" % mean_squared_error(y_train, y_pred))
    print("Coeficiente de determinación R^2 con reg lineal:", regr.score(X, y))
    print("Coeficiente de determinación R^2 con reg logistica:", regr2.score(X, y))
    
    
probando("Normal_Operation","Attacked_Operation","Imbalanced_Attacked",3) 

0    11173
1     3200
Name: Label, dtype: int64
Coefficients: 
 [-1.81905765e+00 -5.74867264e+08  5.74867265e+08  8.42763475e-02
 -3.20460526e+10  3.20460526e+10 -3.22911558e-02  3.84607490e+00
  7.39798348e+08 -7.39798350e+08  9.97191564e-02 -1.87638658e+08
  1.87638658e+08  5.87802352e-02 -3.56903761e+00  1.88043560e+09
 -1.88043560e+09  5.19539755e-02  4.87145562e+10 -4.87145562e+10
  6.36177951e-03  1.50300389e+00 -2.06788969e+09  2.06788969e+09
 -8.40952367e-03 -2.96283224e+10  2.96283224e+10  6.01154474e-03]
Mean squared error: 0.01
Coeficiente de determinación R^2 con reg lineal: 0.9526887232710781
Coeficiente de determinación R^2 con reg logistica: 0.9954080567731163


C:\Users\pazno\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Esto es por si no funciona lo de arriba para escalar
base_aux=pd.DataFrame()
    base_aux2=pd.DataFrame()
    base2_aux2=pd.DataFrame()
    #for i in numerical_consolidated_df.index:
    #for col in numerical_consolidated_df.columns:
    for j in range(1,delta_movil+1):
        base_aux=pd.DataFrame(columns=['dataset_'+str(j)])
        base_aux2=pd.concat([base_aux2,base_aux])
        base_aux3=pd.concat([base_aux2,numerical_consolidated_df])
        
        for col in base_aux3.columns:
            str(col)
            base_aux3.drop([col],axis=1)
        
        
        #base2_aux=pd.DataFrame(columns=['Label_'+str(j)])
        #base2_aux2=pd.concat([base2_aux2,base2_aux])
        #base2_aux3=pd.concat([base2_aux2,numerical_consolidated_df])
        #base_aux4=pd.concat([base2_aux3,base_aux3])
        
        #base_aux2.loc[j-1]=numerical_consolidated_df['dataset_'+str(j)]
            #if col == 'dataset_'+str(j):
                #numerical_consolidated_df=pd.concat([numerical_consolidated_df,numerical_consolidated_df[[col]]],axis=1)
                #auxiliar=numerical_consolidated_df['Label_'+str(j),'dataset_'+str(j)]
                #numerical_consolidated_df.drop([col],axis=1)
    
    return #base_aux3
    
   #invoices.drop(['invoice', 'client', 'units'], axis=1)
    #for col in df.columns:
    #if 'A' in col:
     #   del df[col]
    
    #indexNames = df[ (df['Price'] >= 30)
     #           & (df['Price'] <= 70) ].index
#df.drop(indexNames , inplace=True)
    
# This will return false
# As the value passed is string and you are checking it with int



#extra cuando hice el primer pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
    regr = LinearRegression()
    regr2=LogisticRegression()
    
    regr.fit(X_train, y_train)
    regr2.fit(X_train, y_train)
    
    y_pred = regr.predict(X_train)
    print('Coefficients: \n', regr.coef_)
    from sklearn.metrics import mean_squared_error, r2_score
    print("Mean squared error: %.2f" % mean_squared_error(y_train, y_pred))
    print("Coeficiente de determinación R^2 con reg lineal:", regr.score(X, y))
    print("Coeficiente de determinación R^2 con reg logistica:", regr2.score(X, y))